In [14]:
from datetime import datetime
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm
from pprint import pprint
import pickle
import heapq as heap
import sys
import math

In [3]:
with open('data/graph_total.pkl', 'rb') as f:
    G_total = pickle.load(f)

In [4]:
def interval_time(G, time_inter):
    '''
    :param G: initial graph, with u, v and weights and time_list as attributes
    :param time_inter: [start_date, end_date]
    :return: new_G: graph with only u, v and weight, which is the length of all
                    the dates inside the time interval
    '''
    start = time_inter[0]
    end = time_inter[1]
    new_G = nx.DiGraph()
    for u, v, data in tqdm(G.edges(data=True)):
        time_list = data['time_list']
        # leave only the dates inside the time interval
        new_time_list = [time for time in time_list if start <= time <= end]
        # new weight, which is the length of the new_time_list
        w = len(new_time_list)
        # if the weight differ from 0, then the edge exists
        if w != 0:
            new_G.add_edge(u, v, weight=w)
    return new_G

In [5]:
def dijkstra(df, source, target, G):
    '''
    :param df: dataframe of the graph: source, target, weight
    :param source: source node
    :param target: target node
    :return: final_weight, path
    '''
    # init the distance dictionary with the source node as key and as value None (parent node) and 0 (distance)
    distance_par = {source: (None, 0)}
    visited = set()
    # store the current node
    curr_node = source

    while curr_node != target:
        visited.add(curr_node)
        dest_list = df[df['source']==curr_node].target.to_list()
        # taking only the weight of curr_node
        curr_weight = distance_par[curr_node][1]
        for node in dest_list:
            weight = G[curr_node][node]['weight'] + curr_weight
            if node not in distance_par:
                distance_par[node] = (curr_node, weight)
            else:
                node_weight = distance_par[node][1]
                if node_weight > weight:
                    distance_par[node] = (curr_node, weight)

        # create a list of nodes to visit
        next_dest_list = {n: distance_par[n] for n in distance_par if n not in visited}

        # check if there are nodes to visit
        if not next_dest_list:
            return 'No possible path'

        # next curr node is the one in next_dest_list (nodes still to visit) with the lowest weight
        curr_node = min(next_dest_list, key=lambda k: next_dest_list[k][1])

    # now we wont to compute the list of the shortest path from source to target
    short_path = []
    # at the beginning, curr_node is the target node, since the first while is finished
    final_weight = distance_par[curr_node][1]
    while curr_node is not None:
        short_path.append(curr_node)
        par_node = distance_par[curr_node][0]
        curr_node = par_node

    # reverse the list now
    short_path = short_path[::-1]

    return [final_weight, short_path]

In [6]:
def in_degree(df_g, node):
    # number of edges where node is target
    return df_g[df_g['target']==node].target.count()

def out_degree(df_g, node):
    # number of edges where node is source
    return df_g[df_g['source']==node].source.count()

Given:
- The adjacency matrix $A$
- probability of teleporting $a$

we add links to the nodes that don't have outlinks, and then we normalize each row to build the stochastic matrix $\tilde{A}$.

Then, we make the transition matrix $P$, where the generic element $p_{ij} = Pr$(to go from node i to node j), as 
$$\begin{aligned}
P = \frac{a}{n}\tilde{A}+(1-a)\tilde{A}
\end{aligned}$$
The probability vector at step t is $\vec{q^t}$, where the generic element $q_v^t = Pr$(to be in state v), in general: 
$$\begin{aligned}
\vec{q^t} = \vec{q^{t-1}}P
\end{aligned}$$

After a high number of steps, it converges to the stationary distribution $\vec{\pi}$, corresponding to the (left) eigenvector with eigenvalue 1:  

$$\begin{aligned}
\vec{\pi} = \vec{\pi}P \implies \pi_i = \sum_{j=1}^{n}\pi_jP{ji}
\end{aligned}$$

where $n$ is the number of nodes.

In [7]:
def PageRank(G, a=0.15, max_iter=100, epsilon=1e-6):
    '''
    input:
    G = graph
    a = probability of teleporting
    
    output:
    pi = pagerank score of nodes
    '''
    nodes_list = list(G.nodes)
    n = len(G.nodes) 
    pi = {v:1 for v in nodes_list}
    
    # build adjacency matrix A
    A = np.zeros((n,n))  
    for node in G.nodes:
        i = nodes_list.index(node)
        for neighbor in G.successors(node):
            j = nodes_list.index(neighbor)
            A[i][j] = 1/len(list(G.successors(node)))
    
    #adding an edge to each node for those nodes whose out-degree = 0 and normalize        
    for i in range(n):
        if np.sum(A[i])==0:
            A[i] = (1/n)*np.ones((1, n)) 
    
    # build P matrix
    P = (a/n)*np.ones((n,n))+(1-a)*A
    
    # probability vector initialization
    q = np.zeros((1,n)) 
    q[0][0] = 1
    
    for i in range(max_iter):
        q_new = np.dot(q,P) # q(t+1) = q(t)*P
        
        if epsilon:
            if np.sum(np.abs(q-q_new)) < epsilon:
                break
        
        q = q_new

    for node in nodes_list:
        pi[node] = q[0][nodes_list.index(node)]
    
    return pi

## Functionality 2
We compute the following centrality measures:
- **Betweenness:** $$\begin{align}
Betweenness(v)= \sum_{u\neq v\neq w}\frac{g_{uw}^v}{g_{uw}}
\end{align}$$
Betweenness centrality is defined as the fraction of shortest paths that goes from u to w through v, out of all the shortest paths from u to w.


- **PageRank** of a node depends on the links structure of the graph, and on the importance of the other nodes. We used the random surfer algorithm to compute the pagerank $\pi_v$ of the node $v$. Since $\pi_v$ is the asymptotic probability to be in node $v$, a high value of pagerank means that, after many steps, the random surfer will end up in that node more often than in the others.


- **ClosenessCentrality:** $$\begin{align}
Closeness(v)= \frac{N-1}{\sum_{u}d(u,v)} 
\end{align}$$
where $N$ is the number of nodes in the (connected) graph.
A high value of closeness means that a node is "close" to the others, indeed, the more a node is central, the more is near to the other ones.


- **DegreeCentrality** of a node v is defined as 
$$\begin{align}
degree\_centrality(v) = \frac{degree(v)}{n-1} 
\end{align}$$  where degree(v) is the number of neighbors of v.
If degree centrality of v is high, the node has more links. We usually normalize the degree with the number of edges in the graph (n-1). 



In [15]:
def BestUsers(G, v, metric, time_inter):
    '''
    input:
    G: graph
    v: node of interest
    metric: one centrality measure in Betweenness, PageRank, ClosenessCentrality, DegreeCentrality
    time_inter: [start_date, end_date]
    
    output:
    value of the given metric for node v in graph G in the given interval of time
    '''
    G = interval_time(G_total, time_inter)
    output = None
    
    nodes_list = list(G.nodes)
    df = nx.to_pandas_edgelist(G, nodelist = nodes_list)
    
    shortest_paths = {n:[] for n in G.nodes}
    for n in nodes_list:
        for u in nodes_list:
            if dijkstra(df, n, u, G) != 'No possible path':
                shortest_paths[n].append(dijkstra(df, n, u, G))

    sp_total = 0
    for node in shortest_paths:
        for path in shortest_paths[node]:
            sp_total +=1

    if metric == "Betweenness":
        paths_via_v = 0
        #sp_total = len(shortest_paths[v]) # number of all the shortest paths from node v
        for u in nodes_list: # for each node u in G
            for n_paths in range(len(shortest_paths[u])): # for all paths from u 
                path = shortest_paths[u][n_paths][1]
                if v in path[1:-1]:
                    paths_via_v += 1
                else:
                    continue
                    
        Betweenness = paths_via_v/sp_total/math.comb(len(G.nodes)-1,2)
        output = Betweenness
    
    if metric == "PageRank":
        a = 0.15
        output = PageRank(G, a)[v]
                          
    if metric == "ClosenessCentrality":
        Closeness = 0
        dist = 0
        nodes_v = -1
        for node in shortest_paths:
            for path in shortest_paths[node]:
                if v in path[1]:
                    nodes_v += 1
                    
        for n in G.nodes:
            if dijkstra(df, n, v, G) != 'No possible path':
                dist += dijkstra(df,n,v,G)[0] # number of edges in the shortest path
        
        if dist > 0:
            Closeness = (((nodes_v-1))/dist)
        output = Closeness
    
    if metric == "DegreeCentrality":
        degree = in_degree(df, v) + out_degree(df, v)
        num_nodes = len(nodes_list)
        output = degree/(num_nodes-1) 
                       
    return output 

In [10]:
# We take a time interval of length 1 day to analize importance of the users.
time_inter = ["2015-08-01 ","2015-08-02"]
new_g = interval_time(G_total, time_inter)
nodes_list = list(new_g.nodes)

g = new_g.subgraph(nodes_list[20:80])
nodes_list_g = nodes_list[20:80]
dfg = nx.to_pandas_edgelist(g, nodelist=nodes_list_g)
dfg

100%|██████████| 10300590/10300590 [00:08<00:00, 1260428.20it/s]


,source,target,weight
0,458741,2318263,1
1,1766831,5014875,1
2,1766831,4256636,2
3,1766831,3123921,1
4,1766831,3297613,3
5,1766831,3309077,1
6,1766831,1575725,1
7,4256636,1766831,2
8,4272464,3841809,1
9,3841809,4272464,1


In [16]:
col = ["Betweenness", "PageRank", "ClosenessCentrality", "DegreeCentrality"]
centrality = pd.DataFrame(index = nodes_list_g, columns = col)
for node in nodes_list_g:
    for c in col:
        centrality.loc[node,c] = BestUsers(g, node, c)
centrality

,Betweenness,PageRank,ClosenessCentrality,DegreeCentrality
5181605,0.0,0.00842,0,0.0
458741,0.0,0.00842,0,0.016949
2318263,0.0,0.015577,0.0,0.016949
1766831,0.00002,0.017709,5.5,0.118644
5014875,0.0,0.010929,0.25,0.016949
4256636,0.0,0.010929,3.0,0.033898
3123921,0.0,0.010929,0.25,0.016949
3297613,0.0,0.010929,0.125,0.016949
3309077,0.0,0.010929,0.25,0.016949
1575725,0.0,0.010929,0.25,0.016949


In [44]:
# Maximum values
maxBet = centrality.loc[centrality['Betweenness'] == max(centrality.loc[:,"Betweenness"]),"Betweenness"]
maxPR = centrality.loc[centrality['PageRank'] == max(centrality.loc[:,"PageRank"]),"PageRank"]
maxCC = centrality.loc[centrality['ClosenessCentrality'] == max(centrality.loc[:,"ClosenessCentrality"]),"ClosenessCentrality"]
maxDC = centrality.loc[centrality['DegreeCentrality'] == max(centrality.loc[:,"DegreeCentrality"]),"DegreeCentrality"]

max_values = pd.DataFrame(index = ["User","Value"], 
                          columns = ["Betweenness", "PageRank", "ClosenessCentrality", "DegreeCentrality"])
max_values.iloc[0,0], max_values.iloc[1,0] = maxBet.index[0], maxBet.values[0]
max_values.iloc[0,1], max_values.iloc[1,1] = maxPR.index[0], maxPR.values[0]
max_values.iloc[0,2], max_values.iloc[1,2] = maxCC.index[0], maxCC.values[0]
max_values.iloc[0,3], max_values.iloc[1,3] = maxDC.index[0],maxDC.values[0]

max_values

,Betweenness,PageRank,ClosenessCentrality,DegreeCentrality
User,438154,4272464,1766831,438154
Value,0.000047,0.056131,5.5,0.152542


We can see that user 4272464 has the maximum value of pagerank but it has (approximately) 0 as betweenness centrality. This means that it may have a lot of incoming links, but it doesn't appear in many shortest paths.

Vice versa a node could have high betweenness centrality score (if it appears in many shortest paths, it means that it connects different parts of the graph) but a low PageRank score if it is distant from the centers (node that have more links in the network).

Node 438154 has highest value of both Betweenness and degree centrality, this means that, since it has many links (high degree score), it appears in many shortest paths (high betweenness). 